In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution3D, MaxPooling3D
from keras.utils import np_utils
from keras import backend as K

'''
Creates a keras model with 3D CNNs and returns the model.
'''
def classifier(input_shape, kernel_size, pool_size):
    model = Sequential()

    model.add(Convolution3D(16, kernel_size[0], kernel_size[1], kernel_size[2],
                            border_mode='valid',input_shape=(1, 36, 36, 36)))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(32, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(64, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    return model

def train_classifier(input_shape):
    model = classifier(input_shape, (3, 3, 3), (2, 2, 2))
    model.compile(loss='categorical_crossentropy',
          optimizer='adadelta',
          metrics=['accuracy'])
    '''
    Read the preprocessed datafiles chunk by chunnk and train the model on that batch (trainX, trainY) using:'''
    model.train_on_batch(trainX, trainY, sample_weight=None)
    '''The model can be trained on many epochs using for loops'''
    
    '''
    AFter training the dataset we test our model of the test dataset, read the test file chunk by chunk and 
    test on each chunk (trainX, trainY) using:'''
    print (model.test_on_batch(trainX, trainY, sample_weight=None))
    '''The average of all of this can be taken to obtain the final test score.'''
    
    '''After testing save the model using'''
    model.save('my_model.h5')

In [3]:
def train_generator(folders):
    trainX = np.empty([len(folders)*108, 36, 36, 36])
    trainY = np.empty([len(folders)*108,2])
    for i in tqdm(range(len(folders))):
        X = np.load(output_true + folders[i])
        Y = np.load(output_true + folders[i][:-10] + 'Ytrain.npy')
        if X.shape == (108,36,36,36):
            trainX[i*108:(i+1)*108] = X
            y = np.empty([108,2])
            for j in range(len(Y)):
                if Y[j] == 'True':
                    y[j][0] = 1
                    y[j][1] = 0
                else:
                    y[j][1] = 1
                    y[j][0] = 0


            trainY[i*108:(i+1)*108] = y
    trainX = np.expand_dims(trainX,1)     
    return trainX,trainY

def model_30_cube(input_shape, kernel_size, pool_size):    
    model = Sequential()

    model.add(Convolution3D(16, kernel_size[0], kernel_size[1], kernel_size[2],
                            border_mode='valid',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(32, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(64, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
    return model

def fenlei_fit(name, load_check = False,batch_size=2, epochs=100,check_name = None):

    t = time.time()
    callbacks = [EarlyStopping(monitor='acc', patience = 5, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='acc',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        model = model_30_cube(input_shape, (3, 3, 3), (2, 2, 2))
    x,y = train_generator(folders)
    model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs,
              validation_split = 0.2,verbose=1, callbacks=callbacks, shuffle=True)
    
 
    return model

In [ ]:
output_path = PATH['cls_train_30']
output_true = PATH['cls_train_30_true']
folders = [x for x in os.listdir(output_true) if 'Xtrain' in x]
model_paths = PATH['model_paths']

input_shape = (1, 36, 36, 36)

In [ ]:
fenlei_fit('Fenge_0613', load_check = False, batch_size=2, epochs=20, check_name = 'Fenge_0613')

100%|██████████| 975/975 [07:17<00:00,  1.76it/s]


Train on 84240 samples, validate on 21060 samples
Epoch 1/20
84240/84240 [==============================] - 1985s - loss: nan - acc: 0.0094 - val_loss: nan - val_acc: 0.0092
Epoch 2/20
84240/84240 [==============================] - 2169s - loss: nan - acc: 0.0079 - val_loss: nan - val_acc: 0.0092
Epoch 3/20
70108/84240 [=======================>......] - ETA: 318s - loss: nan - acc: 0.0081